# MNIST Classification
> Performance analysis for MNIST Classification on all hardware platforms

- toc: true 
- badges: true
- comments: true
- categories: [MNIST,Rooflines,Performance Prediction]
- image: images/mnist.png

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

pd.options.display.float_format = '{:20,.0f}'.format
pd.options.display.max_rows =100000 # this will set limit of columns to 500
#master_df.loc[master_df['NN_Topology'] =='MLP']

csv_path = "./data/cleaned_csv/backup.csv"

# Theoretical Analysis of MNIST

### Rooflines for All Hardware Platforms and CNNs

Combining application requirements with hardware platform characteristics can be leveraged for performance predictions using UCB’s roofline models. Using assumptions for where weights, activation tensors, and state of a neural network are stored, combined with the size of the datatypes used, allow us to derive the arithmetic intensity of a neural network during inference. Combined with the roofline for a given hardware platform, we can provide insight as to whether a neural network will be memory or compute bound and guidance for what is theoretically possible in regards to its throughput.

In [2]:
#hide_input

#first process the following csv's to get clean ready-to-plot csv's
%run scripts/script_load_save_data.py
clean_csv_rooflines(path_topologies='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/topology_details.csv',
                    path_hardware='c:/Users/alinav/Documents/GitHub/Qutibench_Web/_notebooks/data/peakPerfBandHardPlatf.csv')

#Now get the cleaned csv, and plot it as a Roofline
%run scripts/altair_plots.py
rooflines(dataframe=pd.read_csv("data/cleaned_csv/rooflines_hardware_neural_networks.csv"), 
          neural_network='mnist')

alt.LayerChart(...)

### Performance Prediction

The following heatmap shows the theoretical performance for the listed hardware platforms for MNIST classification. The metric used for the theoretical performance is input/second.
This plot shows that a combination between prunning and quantization outputs some of the best performance results.

In [3]:
#hide_input
path_csv = 'data/performance_predictions_imagenet_mnist_cifar_2.csv'
    ## Reading csv file and converting data to (Neural network, Platform, Value)
df = pd.read_csv(path_csv)

df1 = pd.DataFrame()
columns = (df.loc[:, df.columns!='hardw']).columns #select all columns except first
for column in columns:
    df_=pd.melt(df, id_vars=['hardw'], value_vars=column) #melt df1 into a df1 of 2 columns
    df1=pd.concat([df1,df_])
df1.columns= ['y','x','values'] #setting new column names
 #replace 0s for NaN values because with 0s the grid doesn't show up
df1['values'] = df1['values'].replace({ 0.0:np.nan})
df_mnist = dataframe_contains(input_df=df1, column='x', value='MLP')
 #Saving above dataframes to csv file
df_mnist.to_csv('c:/Users/alinav/Documents/GitHub/QutibenchWeb/_notebooks/data/cleaned_csv/performance_prediction_mnist.csv', index = False)

heatmap(df_mnist, 'pink', 'Performance prediction for MNIST')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for MNIST 

In this table, within the rows, we show the type of hardware platforms that we used for this task (for example FPGA or GPU) and then more specifically the exact name of the different hardware platforms. For each hardware platform, we list the sweep of specific deployment parameters (batch sizes, operating modes etc) that were used for the experimentation in separate columns. In the columns, we show CNN topologies. When a CNN topology was implemented on a given hardware platform, we show in the corresponding cell the precisions (quantization information) and the channel pruning scale. Otherwise, “na” indicates that the topology wasn’t executed on this specific hardware platform. Many combinations between topology and hardware platform are not supported by the vendors dedicated software environments. INTx depicts a fixed point integer representation with x bits. FPy represents a floating point representation with y bits, for example FP32 is singe precision floating point. Table follows below.

In [4]:
#hide
%run scripts/script_tables.py  #get table with the experiments overview
print(pd.read_csv('data/overview_experiments_mnist.csv').to_markdown())

|    | Hardware   | Platform         | MLP                                 | Batch/Stream/Thread                  |
|---:|:-----------|:-----------------|:------------------------------------|:-------------------------------------|
|  0 | FPGA       | ZCU102-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  1 | FPGA       | ZCU104-DPU       | na                                  | [1,2,3,4,5,6,7,8]                    |
|  2 | FPGA       | Ultra96-DPU      | na                                  | [1,2,3,4,5,6,7,8]                    |
|  3 | FPGA       | ZCU104-FINN      | [INT2, INT4] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128,256,512,10000] |
|  4 | FPGA       | ZCU104-BISMO     | [INT2, INT4] * [100%,50%,25%,12.5%] | [2,4,8,16,32,64,128]                 |
|  5 | GPU        | TX2-maxn         | [FP16, FP32] * [100%,50%,25%,12.5%] | [1,2,4,8,16,32,64,128]               |
|  6 | GPU        | TX2-maxp         | [FP16, FP32] * [100%,50%,25%,12.5

In [5]:
#hide_input
%run scripts/script_tables.py 
dataframes = csv_to_dataframe_multiindex(['data/overview_experiments_mnist_.csv'])
for dataframe in dataframes:   
       display(HTML(dataframe.to_html(index=False)))

In [6]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "TX2" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_datatype_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor]), axis=1)
mnist_df = maxp_df[(maxp_df["NN_Topology"] == "MLP") & maxp_df['lat-comp'].notna() & maxp_df["top1 [%]"].notna()]
mnist_df["hw_datatype_prun"] = mnist_df.apply(lambda r: "_".join([r.HWType, r.Datatype, r.PruningFactor]), axis=1)
mnist_df["PruningFactor"] = mnist_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(mnist_df, "lat-comp", "NN_Topology");
mnist_df["datatype_model"] = mnist_df.Datatype + '_' + mnist_df.HWType
#mnist_df= maxp_df[(maxp_df.NN_Topology == 'MLP') & maxp_df["top1 [%]"].notna()]
mnist_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

#filling GOPS values 
mnist_df['GOPS'] = mnist_df.apply(lambda r: 0.020029 if r.PruningFactor == 100 else 
                                          (0.00582 if r.PruningFactor == 50 else 
                                           (0.001862 if r.PruningFactor == 25 else 0.000669 )) , axis=1)
#fill in tp-system and tp-cmp
mnist_df['tp-system'] = mnist_df['GOPS'] * mnist_df['fps-system']
mnist_df['tp-comp'] = mnist_df['GOPS'] * mnist_df['fps-comp']
mnist_df['GOPS'] = mnist_df['GOPS'] * mnist_df['batch/thread/stream']
#mnist_df.head()

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [7]:
#hide
mnist_df

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,...,top5 [%],Base_Pwr_W,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_datatype_prun,norm-lat-comp,datatype_model
135,MLP,NCS,FP16,NaN,1,4.57,3,219,346,4,...,nan,1,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS
136,MLP,NCS,FP16,NaN,2,8.09,5,247,427,5,...,nan,1,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS
137,MLP,NCS,FP16,NaN,4,14.84,8,269,493,5,...,nan,1,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS
138,MLP,NCS,FP16,NaN,8,28.3,15,283,540,6,...,nan,1,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS
139,MLP,NCS,FP16,NaN,16,54.64,27,293,582,6,...,nan,1,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS
140,MLP,NCS,FP16,NaN,32,107.39,53,298,603,6,...,nan,1,1,2,1,100,l3,NCS_FP16_100%,0,FP16_NCS
141,MLP,NCS,FP16,NaN,64,212.26,104,302,613,6,...,nan,1,1,2,1,100,l3,NCS_FP16_100%,0,FP16_NCS
142,MLP,NCS,FP16,NaN,128,421.84,206,303,621,6,...,nan,1,1,2,3,100,l3,NCS_FP16_100%,0,FP16_NCS
161,MLP,TX2,FP16,maxp,1,1.03,1,976,"1,466",20,...,nan,2,5,10,0,100,l3,TX2_FP16_100%,0,FP16_TX2
162,MLP,TX2,FP16,maxp,2,1.25,1,"1,636","2,786",33,...,nan,2,5,10,0,100,l3,TX2_FP16_100%,0,FP16_TX2


### Line Plot

In [8]:
#hide_input
figa_df = mnist_df[(mnist_df["HWType"].isin(["NCS", "ZCU104-Bismo", "U96-Quadcore A53"]))]
figb_df = mnist_df[(mnist_df["HWType"].isin(["TX2", "ZCU104-FINN", "U96-Quadcore A53"]))]

fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_datatype_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_datatype_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_datatype_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized MLP Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

alt.HConcatChart(...)

### Boxplots

In [9]:
#hide_input
%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="lat-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Latency by Hardware/Framework and Pruning for MLP")

alt.FacetChart(...)

In [10]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="fps-comp", color_col= 'PruningFactor', facet_column='datatype_model' , title="Throughput by Hardware/Framework and Pruning for MNIST Classification")

alt.FacetChart(...)

In [11]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
boxplot(df=mnist_df, xaxis='PruningFactor', yaxis="Full_Pwr_W", color_col= 'PruningFactor', facet_column='datatype_model' , title="Power Consumption by Hardware/Framework and Pruning for MNIST Classification")

alt.FacetChart(...)

### Pareto Graphs

The following pareto graph presents the accuracy versus performance in fps for all the Hardware Platforms across different Pruning and Quantization configurations. This provides insights into accuracy-based comparisons.

In [12]:
#hide_input
#%run scripts/altair_plots.py  #run the plot script if it wasn't previously run
pareto_graph(df= mnist_df, 
             groupcol= 'hw_datatype_prun',
             xcol = 'fps-comp', 
             ycol= 'top1',
             W= W, 
             H= H, 
             title= "MNIST Cassification Design Space: Accuracy versus Performance")

alt.LayerChart(...)

In [13]:
#hide
def replace_data_df(df_: pd.DataFrame(), column:str, list_tuples_data_to_replace: list )-> pd.DataFrame():
    """Method to replace a substring inside a cell inside a dataframe"""
    df = df_.copy()
    for j, k in list_tuples_data_to_replace:
        df[column] = df[column].str.replace(j, k)
    return df

In [14]:
#hide
""" This is to create a df to be joined with the theoretical df in 'Theoretical Analysis' to create the overlapped paretos

Steps
------
1. Create subset from imagenet that doesn't have the ResNet50 v15 measurements because it does not have accuracy measures
2. Create new hardware column that has hardware and operation mode, beware with NaNs
3. Create new 'hardw_datatype_net_prun' with hardware + datatype + netwrok + pruning
4. Create a suset of the dataframe with the above mentioned column and the corresponding ones
5. With groupby for col 'hardw_datatype_net_prun', for each unique value get the rows with biggest batch 
6. Add 'type column', reset the index from 'hardw_datatype_net_prun' to ints and save it
"""
#This is to get the pareto plot for biggest batch size
#Create mnist_df_tmp from mnist_df
mnist_df_tmp = mnist_df
#   create hardw column to include: hardware + op_mode
mnist_df_tmp['hardw']= mnist_df_tmp['HWType'] + ('-' + mnist_df_tmp['Op mode']).fillna('')
#   create hardw_datatype_net_prun col with all those columns merged
mnist_df_tmp['hardw_datatype_net_prun'] = mnist_df_tmp.apply(lambda r: "_".join([r.hardw, r.Datatype,r.NN_Topology, str(r.PruningFactor)]), axis=1)
#   create a subset of the dataframe with only those columns
mnist_df_tmp = mnist_df_tmp[['hardw_datatype_net_prun', 'hardw','NN_Topology','fps-comp', 'top1','batch/thread/stream']]
#   Only get the points corresponding to the biggest batch
mnist_df_tmp = mnist_df_tmp.groupby('hardw_datatype_net_prun')[['batch/thread/stream', 'hardw','NN_Topology','fps-comp', 'top1']].max()
#   add type column
mnist_df_tmp['type'] = 'measured'
#   reset index to start being numeric 
mnist_df_tmp = mnist_df_tmp.reset_index()
#   change column names
mnist_df_tmp.columns = ['hardw_datatype_net_prun', 'batch/thread/stream', 'hardw', 'network', 'fps-comp', 'top1', 'type']
#   fix samll stuff in the df so things match with the other side
mnist_df_tmp = replace_data_df(df_=mnist_df_tmp, column='hardw_datatype_net_prun', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1'),('100.0','100'),('25.0','25') ,('50.0','50')])
mnist_df_tmp = replace_data_df(df_=mnist_df_tmp, column='network', list_tuples_data_to_replace=[("RN50", "ResNet50"),("MNv1", "MobileNetv1"),('GNv1','GoogLeNetv1')])
#delete unnecessary columns
mnist_df_tmp = mnist_df_tmp.drop(columns=['batch/thread/stream'])

#save it all
mnist_df_tmp.to_csv('data/cleaned_csv/pareto_data_mnist.csv', index = False)
#plot the pareto to confirm all is good
pareto_graph(df= mnist_df_tmp, 
             groupcol= 'hardw_datatype_net_prun', 
             xcol= 'fps-comp', 
             ycol= 'top1', 
             W= W, 
             H= H, 
             title= "MNIST Cassification Design Space: Accuracy versus Performance")
mnist_df_tmp

C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\alinav\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,hardw_datatype_net_prun,hardw,network,fps-comp,top1,type
0,NCS_FP16_MLP_100,NCS,MLP,621,99,measured
1,NCS_FP16_MLP_12.5,NCS,MLP,"2,796",98,measured
2,NCS_FP16_MLP_25,NCS,MLP,"2,197",98,measured
3,NCS_FP16_MLP_50,NCS,MLP,"1,371",99,measured
4,TX2-maxp_FP16_MLP_100,TX2-maxp,MLP,"61,409",97,measured
5,TX2-maxp_FP16_MLP_12.5,TX2-maxp,MLP,"278,034",97,measured
6,TX2-maxp_FP16_MLP_25,TX2-maxp,MLP,"220,832",97,measured
7,TX2-maxp_FP16_MLP_50,TX2-maxp,MLP,"141,671",97,measured
8,TX2-maxp_FP32_MLP_100,TX2-maxp,MLP,"39,802",97,measured
9,TX2-maxp_FP32_MLP_12.5,TX2-maxp,MLP,"264,873",97,measured


# Theoretical Pareto and Measured Pareto Overlapped

In order to easily understand how accurate predictions were, an overlapping between the Theoretical Pareto Plot and Measured Pareto Plot was made. In the plot below we have both theoretical (orange) and measured (blue) pareto lines. All measured datapoins are represented as crosses and all theoretical datatpoins are represented as circles. Some theoretical datapoints don't have a measured matched datapoint and the same goes for the measured datapoints. The theoretical pareto curve is, as expected, on the right of the measured one, as predictions are sometimes different form measurements.

In [15]:
#hide_input
%run scripts/overlapped_pareto.py
get_overlapped_pareto('mnist')

alt.LayerChart(...)

# Efficiency Plot

In order to understand the gap between the theoretical predicitons and what was measured an Efficiency Barchart was created.
In blue we have all thoretical predictions and in orange we have all measured datapoints. Some percentage values might seem a bit strange due to the logarihmic y axis scale. 

In [18]:
#hide_input
%run scripts/overlapped_pareto.py
efficiency_plot(net_keyword= 'mnist', title='Efficiency plots for MNIST')

alt.FacetChart(...)

In [17]:
#hide
mnist_df.to_csv('data/cleaned_csv/experimental_data_mnist.csv', index = False)
mnist_df.head(200)

,NN_Topology,HWType,Datatype,Op mode,batch/thread/stream,lat-sys,lat-comp,fps-system,fps-comp,tp-system,...,Idle_Pwr_W,Full_Pwr_W,GOPS,PruningFactor,level,hw_datatype_prun,norm-lat-comp,datatype_model,hardw,hardw_datatype_net_prun
135,MLP,NCS,FP16,NaN,1,4.57,3,219,346,4,...,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
136,MLP,NCS,FP16,NaN,2,8.09,5,247,427,5,...,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
137,MLP,NCS,FP16,NaN,4,14.84,8,269,493,5,...,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
138,MLP,NCS,FP16,NaN,8,28.3,15,283,540,6,...,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
139,MLP,NCS,FP16,NaN,16,54.64,27,293,582,6,...,1,2,0,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
140,MLP,NCS,FP16,NaN,32,107.39,53,298,603,6,...,1,2,1,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
141,MLP,NCS,FP16,NaN,64,212.26,104,302,613,6,...,1,2,1,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
142,MLP,NCS,FP16,NaN,128,421.84,206,303,621,6,...,1,2,3,100,l3,NCS_FP16_100%,0,FP16_NCS,NCS,NCS_FP16_MLP_100.0
161,MLP,TX2,FP16,maxp,1,1.03,1,976,"1,466",20,...,5,10,0,100,l3,TX2_FP16_100%,0,FP16_TX2,TX2-maxp,TX2-maxp_FP16_MLP_100.0
162,MLP,TX2,FP16,maxp,2,1.25,1,"1,636","2,786",33,...,5,10,0,100,l3,TX2_FP16_100%,0,FP16_TX2,TX2-maxp,TX2-maxp_FP16_MLP_100.0
